### correlation

In [1]:
import numpy as np
from scipy.stats import norm
import sys
import os
import importlib

# Add parent folder (one level up from notebook) to Python path
sys.path.append(os.path.abspath(".."))


# If your files aren't named exactly like this, change the names below:
VEC_MOD_NAME = "amp_denoising_functions_vectorized"       # e.g., vectorized.py
NONVEC_MOD_NAME = "lambda_nu_integrals"  # e.g., non_vectorized.py

vec = importlib.import_module(VEC_MOD_NAME)
base = importlib.import_module(NONVEC_MOD_NAME)


# --- Define random variables ---
N = 1_000_000
lambda_ = np.random.randn(N)          # standard Gaussian
nu = np.sign(lambda_)                 # sign(lambda)

# --- Empirical expectation ---
empirical = np.mean(lambda_ * nu)

# --- Theoretical value ---
theoretical = base.corr_int(1)

print(f"Empirical  E[λν] = {empirical:.5f}")
print(f"Theoretical E[λν] = {theoretical:.5f}")
print(f"Error = {abs(empirical - theoretical):.2e}")


Empirical  E[λν] = 0.79769
Theoretical E[λν] = 0.79788
Error = 1.99e-04


G_out test comparing with general dfOut

In [2]:
# parameters
beta_u = 1.0
beta_v = 2.0
gamma = 1.0
delta = 0.0
n = 1000
eps=1e-6

Q = np.eye(2)
V = [np.eye(2) for _ in range(n)]
V = np.array(V)
omega = np.zeros((n,2))
y = np.ones(n)

bt = vec.beta_tilde(beta_v)
c2 = vec.c_2(bt, Q[1,1])
c1 = vec.c_1(c2, bt)
M, N, V_inv, gamma_mat = vec.compute_M_N_from_V_c1(V, c1, eps)
a, b, c, d = vec.integral_parameters(beta_u, beta_v, omega, Q, N, M, c1, c2)

fout = vec.f_out(beta_u, beta_v, gamma, delta, gamma_mat, c1, c2, y, omega, Q, N, a, b, c, d)
dfout = vec.df_out(fout, beta_u, beta_v, gamma, delta, gamma_mat, c1, c2, y, omega, Q, N, a, b, c, d)
Gout = vec.G_out(beta_u, beta_v, gamma, bt)



In [7]:
import numpy as np

# ---- parameter grids (tweak as you like) ----
beta_u_grid = [0.5, 1.0, 2.0]
beta_v_grid = [0.5, 2.0, 5.0]
gamma_grid  = [0.0, 0.5, 1.0]
delta_grid  = [0.0]              # add more if needed
n_grid      = [200, 1000]
eps = 1e-6

rtol = 1e-4
atol = 1e-6

def run_case(beta_u, beta_v, gamma, delta, n, verbose=False):
    # Fixed identities / zeros as specified
    Q = np.eye(2)
    V = np.repeat(np.eye(2)[None, ...], n, axis=0)   # (n,2,2)
    omega = np.zeros((n, 2))
    y = np.ones(n, dtype=int)                         # all active
    
    # Compute all pieces (your functions in `vec`)
    bt = vec.beta_tilde(beta_v)
    c2 = vec.c_2(bt, Q[1,1])
    c1 = vec.c_1(c2, bt)
    M, N, V_inv, gamma_mat = vec.compute_M_N_from_V_c1(V, c1, eps)
    a, b, c, d = vec.integral_parameters(beta_u, beta_v, omega, Q, N, M, c1, c2)

    # f_out for df_out call
    fout = vec.f_out(beta_u, beta_v, gamma, delta, gamma_mat, c1, c2, y, omega, Q, N, a, b, c, d)

    # Evaluate both objects
    df = vec.df_out(fout, beta_u, beta_v, gamma, delta, gamma_mat, c1, c2, y, omega, Q, N, a, b, c, d)  # (n,2,2)
    G  = vec.G_out(beta_u, beta_v, gamma, bt)                     
    
    # print("df")
    # print(df)
    # print("G")
    # print(G)                                      # (2,2)

    # Compare per-sample on active mask
    mask = (y == 1)
    df_active = df[mask]  # (n_active, 2, 2)
    ok = np.allclose(df_active, G[None, :, :], rtol=rtol, atol=atol)
    max_abs_diff = np.max(np.abs(df_active - G[None, :, :])) if df_active.size else 0.0

    if verbose and df_active.shape[0] > 0:
        print("First sample df_out:\n", df_active[0])
        print("G_out:\n", G)
        print("Diff (first sample):\n", df_active[0] - G)

    return ok, max_abs_diff

# ---- run the sweep and print a concise report ----
results = []
for beta_u in beta_u_grid:
    for beta_v in beta_v_grid:
        for gamma in gamma_grid:
            for delta in delta_grid:
                for n in n_grid:
                    ok, diff = run_case(beta_u, beta_v, gamma, delta, n, verbose=False)
                    results.append((ok, diff, beta_u, beta_v, gamma, delta, n))
                    status = "PASS" if ok else "FAIL"
                    print(f"{status} | bu={beta_u:>4}, bv={beta_v:>4}, γ={gamma:>3}, δ={delta:>3}, n={n:>4} | max|Δ|={diff:.3e}")

# Optionally assert all pass:
assert all(r[0] for r in results), "Some parameter sets failed the G_out vs df_out equality."


PASS | bu= 0.5, bv= 0.5, γ=0.0, δ=0.0, n= 200 | max|Δ|=2.500e-07
PASS | bu= 0.5, bv= 0.5, γ=0.0, δ=0.0, n=1000 | max|Δ|=2.500e-07
PASS | bu= 0.5, bv= 0.5, γ=0.5, δ=0.0, n= 200 | max|Δ|=2.500e-07
PASS | bu= 0.5, bv= 0.5, γ=0.5, δ=0.0, n=1000 | max|Δ|=2.500e-07
PASS | bu= 0.5, bv= 0.5, γ=1.0, δ=0.0, n= 200 | max|Δ|=2.500e-07
PASS | bu= 0.5, bv= 0.5, γ=1.0, δ=0.0, n=1000 | max|Δ|=2.500e-07
PASS | bu= 0.5, bv= 2.0, γ=0.0, δ=0.0, n= 200 | max|Δ|=4.444e-07
PASS | bu= 0.5, bv= 2.0, γ=0.0, δ=0.0, n=1000 | max|Δ|=4.444e-07
PASS | bu= 0.5, bv= 2.0, γ=0.5, δ=0.0, n= 200 | max|Δ|=4.444e-07
PASS | bu= 0.5, bv= 2.0, γ=0.5, δ=0.0, n=1000 | max|Δ|=4.444e-07
PASS | bu= 0.5, bv= 2.0, γ=1.0, δ=0.0, n= 200 | max|Δ|=4.444e-07
PASS | bu= 0.5, bv= 2.0, γ=1.0, δ=0.0, n=1000 | max|Δ|=4.444e-07
PASS | bu= 0.5, bv= 5.0, γ=0.0, δ=0.0, n= 200 | max|Δ|=6.944e-07
PASS | bu= 0.5, bv= 5.0, γ=0.0, δ=0.0, n=1000 | max|Δ|=6.944e-07
PASS | bu= 0.5, bv= 5.0, γ=0.5, δ=0.0, n= 200 | max|Δ|=6.944e-07
PASS | bu= 0.5, bv= 5.0, 

In [5]:
run_case(beta_u=3, beta_v=2, gamma=1, delta=0, n=10)

df
[[[ 3.00000900e+00  1.12838011e+00]
  [ 1.12838011e+00 -4.44443852e-07]]

 [[ 3.00000900e+00  1.12838011e+00]
  [ 1.12838011e+00 -4.44443852e-07]]

 [[ 3.00000900e+00  1.12838011e+00]
  [ 1.12838011e+00 -4.44443852e-07]]

 [[ 3.00000900e+00  1.12838011e+00]
  [ 1.12838011e+00 -4.44443852e-07]]

 [[ 3.00000900e+00  1.12838011e+00]
  [ 1.12838011e+00 -4.44443852e-07]]

 [[ 3.00000900e+00  1.12838011e+00]
  [ 1.12838011e+00 -4.44443852e-07]]

 [[ 3.00000900e+00  1.12838011e+00]
  [ 1.12838011e+00 -4.44443852e-07]]

 [[ 3.00000900e+00  1.12838011e+00]
  [ 1.12838011e+00 -4.44443852e-07]]

 [[ 3.00000900e+00  1.12838011e+00]
  [ 1.12838011e+00 -4.44443852e-07]]

 [[ 3.00000900e+00  1.12838011e+00]
  [ 1.12838011e+00 -4.44443852e-07]]]
G
[[ 3.00000000e+00  1.12837917e+00]
 [ 1.12837917e+00 -1.11022302e-16]]


(False, 9.000026999661515e-06)